# Red neuronal multi usos

#### Este código es la estructura basica de una red neuronal de dos capas la cual se puede adecuar para distintos usos solo para fines se prueba se le alimentaran los datos de MNIST, para poderla usar con otros set de datos solo cambia la data de entrada (dependiendo de las variables que vayas a manejar) y las salidas (numero de clasificaciones)

In [6]:
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
#Parametros de entrenamiento de nuestra red

numero_pasos = 500 #El numero de pasos (iteraciones) que se llevaran a cabo en el entrenamiento

lotes = 128 #Cada lote de entrenamiento de cuantos elementos va a ser

taza_aprendizaje = 0.01 #Que tan rapido aprende el algoritmo 

display_step = 100 #Cada cuantos pasos de entrenamiento nos desplegara la terminal como va el aprendizaje del programa

In [4]:
#Parametros de la estructura de nuestra red

numero_entradas = 784 # Numero de entrada que tendremos para nuestra primera capa 

neuronas_capa1 = 100 # Numero de neuronas en nuestra primera capa

neuronas_capa2 = 100 # Numero de neuronas en la segunda capa

numero_clasificaciones = 10 # Numero de clasificaciones en nuestra ultima capa

In [7]:
#Variables que manejaran las entradas y saliedas de nuestra red

X = tf.placeholder("float", [None, numero_entradas])
Y = tf.placeholder("float", [None, numero_clasificaciones])

In [8]:
#Hacemos dos diccionarios, uno con los pesos que usara nuestra red y el otro con los sesgos o bias
diccionario_pesos_W = \
    {
    'entrada_capa1': tf.Variable(tf.random_normal([numero_entradas, neuronas_capa1])), #pesos de la entrada a la primera capa
    'capa1_capa2': tf.Variable(tf.random_normal([neuronas_capa1, neuronas_capa2])), #pesos de primera a segunda capa 
    'capa2_salida': tf.Variable(tf.random_normal([neuronas_capa2, numero_clasificaciones])) #pesos de segunda capa a salida
    }
diccionario_sesgos_b = \
    {
    'biascapa1': tf.Variable(tf.random_normal([neuronas_capa1])), #sesgo de primera capa
    'biascapa2': tf.Variable(tf.random_normal([neuronas_capa2])), #seso de segunda capa
    'biasultimacapa': tf.Variable(tf.random_normal([numero_clasificaciones])) #sesgo de ultima capa 
    }


In [55]:
#Funcion la cual une todo lo que hicimos en los ultimos pasos para ya tener una estructura armada 
def armar_red_neuronal(x):
    # Hidden fully connected layer with 256 neurons
    capa1 = tf.add(tf.matmul(x, diccionario_pesos_W['entrada_capa1']), diccionario_sesgos_b['biascapa1'])
    # Hidden fully connected layer with 256 neurons
    capa2 = tf.add(tf.matmul(capa1, diccionario_pesos_W['capa1_capa2']), diccionario_sesgos_b['biascapa2'])
    # Output fully connected layer with a neuron for each class
    capaSalida = tf.matmul(capa2, diccionario_pesos_W['capa2_salida']) + diccionario_sesgos_b['biasultimacapa']
    return capaSalida

In [56]:
#Construimos la red y hacemos que la ultima capa pase por una funcion de softmax para hacer la clasificacion
logits = armar_red_neuronal(X) #Le alimentamos X ya que es la variable que utilizaremos para la primera capa
prediccion = tf.nn.softmax(logits) #Cual es la prediccion de nuestra red (Que clasificacion)

In [57]:
#Creamos las funciones las cuales se encargan de hacer que nuestro algoritmo aprenda
perdida = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizador = tf.train.AdamOptimizer(learning_rate=taza_aprendizaje)
minimizador = optimizador.minimize(perdida)

In [58]:
#Funciones para medir que tan bien esta aprendiendo el programa durante su entrenamiento
predicciones = tf.equal(tf.argmax(prediccion, 1), tf.argmax(Y, 1))
certeza = tf.reduce_mean(tf.cast(predicciones, tf.float32))

In [59]:
#inicializamos nuestras variables
init = tf.global_variables_initializer()

In [68]:
#Llevamos a cabo el entrenamiento de nuestro modelo
with tf.Session() as sess:

    sess.run(init)

    for step in range(1, numero_pasos + 1):
        batch_x, batch_y = mnist.train.next_batch(lotes)
        
        sess.run(minimizador, feed_dict={X: batch_x, Y: batch_y}) #Correr una paso de entrenamiento
        
        #Todo esto nos enseña que tan bien esta aprendiendo el programa cada 100 pasos de entrenamiento
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([perdida, accuracy], feed_dict={X: batch_x,Y: batch_y})
            print("Paso "+str(step)+ ",Perdida= "+"{:.4f}".format(loss)+", Certeza= "+"{:.3f}".format(acc))


    print("Porcentaje de certeza: {:.2f}" .format(sess.run(certeza, feed_dict=
                                                           {X: mnist.test.images,Y: mnist.test.labels})*100),'%')

Paso 1,Perdida= 718.1907, Certeza= 0.211
Paso 100,Perdida= 71.7677, Certeza= 0.797
Paso 200,Perdida= 20.2565, Certeza= 0.914
Paso 300,Perdida= 40.0049, Certeza= 0.812
Paso 400,Perdida= 13.0502, Certeza= 0.891
Paso 500,Perdida= 20.2619, Certeza= 0.883
Porcentaje de certeza: 88.66 %
